[![DOI](https://zenodo.org/badge/940091341.svg)](https://doi.org/10.5281/zenodo.14939868) <a href="https://www.globh2e.org.au/"><img src="https://img.shields.io/badge/ARC:Funding%20number-IC200100023-blue.svg"/></a>

# **Technology choice model**
<!-- **<div style="text-align: left; font-size: 19px;"> Prospective life cycle assessment data generation**</div> -->

<div style="text-align: left; font-size: 16px;">Michaël Lejeune<sup>a,b</sup>, Sami Kara<sup>a,b</sup>, Michael Zwicky Hauschild<sup>c,d</sup>, Sareh Sharabifarahni<sup>a</sup>, Rahman Daiyan<sup>b,e</sup></div><br>

<div style="text-align: left; font-size: 13px;"><sup>a</sup>Sustainability in Manufacturing and Life Cycle Engineering Research Group, School of Mechanical and Manufacturing Engineering, the University of New South Wales, 2052, Sydney, Australia</div>

<div style="text-align: left; font-size: 13px;">
<sup>b</sup>Australian Research Council Training Centre for the Global Hydrogen Economy (GlobH2e), the University of New South Wales, 2052, Sydney, Australia</div>

<div style="text-align: left; font-size: 13px;">
<sup>c</sup>Centre for Absolute Sustainability, Technical University of Denmark, Kgs, Lyngby, Denmark</div>

<div style="text-align: left; font-size: 13px;">
<sup>d</sup>Division for Quantitative Sustainability Assessment (QSA), Department of Environmental and Resource Engineering, Technical University of Denmark, Kgs, Lyngby, Denmark</div>

<div style="text-align: left; font-size: 13px;">
<sup>e</sup>School of Minerals and Energy Engineering, The University of New South Wales, Sydney 2052, Australia</div><br>

<div style="text-align: left; font-size: 13px;"> Code developer and maintainer:<a href="mailto:m.lejeune@unsw.edu.au"> Michaël Lejeune</div></a><br>


# **1. Introduction**

In [1]:
include("./Utils/1_00_utils.jl")
include("./Utils/3_03_utils.jl")
include("./Utils/3_00_utils.jl")
include("./lca/src/lca.jl")
# include("./plot.jl")
using .LCAModule
using JuMP, CPLEX,Distributions
using PyPlot


┌ Warning: This file was written in a newer version of the JLD2 file format.
│ Please consider updating JLD2.
└ @ JLD2 /Users/mickael/.julia/packages/JLD2/bfPAo/src/file_header.jl:37
┌ Warning: This file was written with a different version of JLD2 that may not be compatible.
│ Attempting to load data.
└ @ JLD2 /Users/mickael/.julia/packages/JLD2/bfPAo/src/file_header.jl:41


In [2]:

# using PyPlot
# fig,ax=pbplot(ASR1[1][1:end-1],categories=categories[1:end-1])
# display(fig)


# **2. Background**

In this section, we demonstrate how the technology choice model operates to provide a comprehensive understanding of its structure and functionality. We utilize the model from Bachmann et al.†, as it offers a clear and systematic framework for readers to grasp the key concepts. The model is adapted to suit the specific requirements of this study.

## **2.1 Review of the different optimisation models**

Kätelhön, Bardow and Suh, (2016)

$$
\min_{\text{s.t.}\ s} Z = \boldsymbol{k^T F s} 
\tag{1}
$$

$$
\boldsymbol{As=f} 
\tag{2}
$$


$$
\boldsymbol{s^*\geq 0} 
\tag{3}
$$

$$
\boldsymbol{F s \leq c} 
\tag{4}
$$

Bachmann et al. 2023

$$
\min_{\text{s.t.}\ s} PF_{CC} = \boldsymbol{q_{CC}^T B s} 
\tag{1}
$$

$$
\boldsymbol{As=f} 
\tag{2}
$$


$$
\boldsymbol{s\geq 0} 
\tag{3}
$$

$$
\boldsymbol{d \leq c} 
\tag{4}
$$

Letchemberg et al. 2024

$$
\min_{s, \text{slack}} w^T z
\tag{1}
$$

$$
A s = f + \text{slack}
\tag{2}
$$

$$
s^{\text{low}} \leq s \leq s^{\text{high}}
\tag{3}
$$

$$
0 \leq \text{slack} \leq \text{slack}^{\text{high}}
\tag{4}
$$

$$
z = Q B s
\tag{5}
$$

$$
z \leq z^{\text{high}}
\tag{6}
$$

$$
B s \leq b^{\text{high}}
\tag{7}
$$


In this research, we implement the following model

$$
\min_{\text{s.t.}\ s} \ \boldsymbol{x  = \Gamma d} 
\tag{1}
$$


$$
\boldsymbol{d = \left(\Lambda s\right)\oslash \omega = \left(Q B s\right)\oslash \omega}
\tag{2}
$$

$$
\boldsymbol{A s = f}
\tag{2}
$$

$$
\boldsymbol{s^* \geq 0}
\tag{2}
$$

$$
\boldsymbol{F s \leq c}
\tag{2}
$$

$$
\boldsymbol{x \leq x^{\text{high}}}
\tag{2}
$$

$$
\boldsymbol{Bs \leq g^{\text{high}}}
\tag{2}
$$


# **3. Rice farming case study**

## **3.0 Data curation**

In [3]:
b = 1:length(processes) # here n refers to the columns of matrix A, which are the processes involved. 
α = 1e-5 # allocation factor
ω = α*Δxᵖᵇ


9-element Vector{Float64}:
 1.0e-5
 6.88e-6
 1.1e-6
 0.04
 0.0001
 0.00039700000000000005
 0.000145
 0.00025
 0.0001

## **3.1 Optimisation**

### **3.1.1 Balanced optimsation**

In [4]:
# import Base: ⊘ # Hadamard's elementary division operator
⊘(a, b) = a ./ b


⊘ (generic function with 1 method)

In [28]:
𝐁 =cat([B,B],dims=3)


2×1×1 Array{Matrix{Float64}, 3}:
[:, :, 1] =
 [0.0 0.061426004 … 1.095184111 0.057554402; 0.0 0.000132928 … 0.000915211 6.96793e-5; … ; 0.0 1000.0 … 0.0 0.0; 0.0 0.006 … 0.0 0.0]
 [0.0 0.061426004 … 1.095184111 0.057554402; 0.0 0.000132928 … 0.000915211 6.96793e-5; … ; 0.0 1000.0 … 0.0 0.0; 0.0 0.006 … 0.0 0.0]

In [5]:
model = Model(CPLEX.Optimizer)
set_silent(model)

# Variables and expressions
@variable(model, 𝐬[1:length(processes)])

@expression(model, 𝐠, B*𝐬)
@expression(model, 𝐝, Q*𝐠 ⊘ ω)
@expression(model, 𝐱, 𝐝)

@objective(model, Min, 𝐱);
@constraint(model, 𝐬 ≥ 0)
@constraint(model, A * 𝐬 == f)
# @constraint(model, F * 𝐬 ≤ c)
optimize!(model);
if !is_solved_and_feasible(model)
    error("Solver did not find an optimal solution")
end
# solution_summary(model)
DataFrame(hcat([catnames_ticks,value.(𝐝),value.(𝐱)]...),["Boundaries","𝐝", "𝐱"])


Row,Boundaries,𝐝,𝐱
,Any,Any,Any
1,Climate change Energy imbalance,6.75738,6.75738
2,Ocean acidification,1.97176,1.97176
3,Atmospheric aerosol loading,1.47063,1.47063
4,Freshwater use,11.5038,11.5038
5,Biogeochemical flows-P,2.62535,2.62535
6,Biogeochemical flows-N,2.89673,2.89673
7,Stratospheric ozone depletion,0.189737,0.189737
8,Land-system change,0.00215625,0.00215625
9,Biosphere Integrity,8.68749,8.68749


In [6]:
DataFrame(hcat(processes,[value.(i) for i ∈ [𝐬]]...),["processes","𝐬"])


Row,processes,𝐬
,Any,Any
1,Rice factory,1.0
2,Rice farming,0.0
3,Low-nitrogen,1.15
4,Rice husk boiler,2.2
5,Natural gas boiler,0.0
6,Wood pellet boiler,0.0
7,Rice husk collection 1,0.506
8,Rice husk collection 2,0.0
9,Rice husk collection 3,0.0


### **3.1.2 Climate change oriented optimisation**

In [7]:
modelᶜᶜ = Model(CPLEX.Optimizer);
set_silent(modelᶜᶜ)

@variable(modelᶜᶜ, 0 ≤ 𝐬ᶜᶜ[1:length(processes)])

@expression(modelᶜᶜ, 𝐠, B*𝐬ᶜᶜ)
@expression(modelᶜᶜ, 𝐝ᶜᶜ, Q[1,:]'*𝐠./ω)
@expression(modelᶜᶜ, 𝐱ᶜᶜ, 𝐝ᶜᶜ)

@objective(modelᶜᶜ, Min, 𝐱ᶜᶜ)
@constraint(modelᶜᶜ , A * 𝐬ᶜᶜ == f) 
# @constraint(modelᶜᶜ , F * 𝐬ᶜᶜ ≤ c)

# @constraint(modelᶜᶜ , ((Λ°[7,:]'*𝐬ᶜᶜ)/ω[7]) ≤ 6)

optimize!(modelᶜᶜ);
solution_summary(model)
𝐬°ᶜᶜ= value.(𝐬ᶜᶜ)
𝐝°ᶜᶜ = value.(𝐝ᶜᶜ)
𝐱°ᶜᶜ = value.(𝐱ᶜᶜ)
DataFrame(hcat([catnames_ticks,𝐝°ᶜᶜ,𝐱°ᶜᶜ]...),["Boundaries","𝐝ᶜᶜ", "𝐱ᶜᶜ"])


Row,Boundaries,𝐝ᶜᶜ,𝐱ᶜᶜ
,Any,Any,Any
1,Climate change Energy imbalance,5.13588,5.13588
2,Ocean acidification,7.46495,7.46495
3,Atmospheric aerosol loading,46.6898,46.6898
4,Freshwater use,0.00128397,0.00128397
5,Biogeochemical flows-P,0.513588,0.513588
6,Biogeochemical flows-N,0.129367,0.129367
7,Stratospheric ozone depletion,0.354199,0.354199
8,Land-system change,0.205435,0.205435
9,Biosphere Integrity,0.513588,0.513588


In [8]:
DataFrame(hcat(processes,[value.(i) for i ∈ [𝐬ᶜᶜ]]...),["processes","𝐬"])


Row,processes,𝐬
,Any,Any
1,Rice factory,1.0
2,Rice farming,1.15
3,Low-nitrogen,0.0
4,Rice husk boiler,2.2
5,Natural gas boiler,0.0
6,Wood pellet boiler,0.0
7,Rice husk collection 1,0.506
8,Rice husk collection 2,0.0
9,Rice husk collection 3,0.0


### **3.1.3 optimisation with informed by biophysical interactions**

#### **3.1.3.1 Regular optimisation**

In [9]:
modelᵦ = Model(CPLEX.Optimizer)
set_silent(modelᵦ)

@variable(modelᵦ, 0 ≤ 𝐬ᵦ[1:length(processes)])

@expression(modelᵦ, 𝐠,  B*𝐬ᵦ)
@expression(modelᵦ, 𝐝ᵦ, Q*𝐠./ω)
@expression(modelᵦ, 𝐱ᵦ, 𝚪ᵦ * 𝐝ᵦ)

@constraint(modelᵦ, A * 𝐬ᵦ == f)
# @constraint(modelᵦ, F * 𝐬ᵦ ≤ c)

@objective(modelᵦ, Min, 𝐱ᵦ)
optimize!(modelᵦ)
solution_summary(modelᵦ)

DataFrame(hcat(catnames_ticks,[value.(i) for i ∈ [𝐝ᵦ,𝐱ᵦ]]...),["Boundaries","𝐝ᵦ", "𝐱ᵦ"])


Row,Boundaries,𝐝ᵦ,𝐱ᵦ
,Any,Any,Any
1,Climate change Energy imbalance,6.75738,11.599
2,Ocean acidification,1.97176,8.26313
3,Atmospheric aerosol loading,1.47063,-5.02481
4,Freshwater use,11.5038,16.702
5,Biogeochemical flows-P,2.62535,10.8017
6,Biogeochemical flows-N,2.89673,10.8017
7,Stratospheric ozone depletion,0.189737,-1.08615
8,Land-system change,0.00215625,6.1817
9,Biosphere Integrity,8.68749,16.4383


In [10]:
DataFrame(hcat(processes,[value.(i) for i ∈ [𝐬ᵦ]]...),["processes","𝐬ᵦ"])


Row,processes,𝐬ᵦ
,Any,Any
1,Rice factory,1.0
2,Rice farming,0.0
3,Low-nitrogen,1.15
4,Rice husk boiler,2.2
5,Natural gas boiler,0.0
6,Wood pellet boiler,0.0
7,Rice husk collection 1,0.506
8,Rice husk collection 2,0.0
9,Rice husk collection 3,0.0


#### **3.1.3.2 Stochastic optimisation**

In [11]:

@load "../Source data/02_results//1_00_total_human_impact/data_interaction_param_matrices.jld2" δ𝐁


1-element Vector{Symbol}:
 :δ𝐁

In [12]:
σω = 0.001 .* abs.(ω) # standard deviation for the perturbed vector ω
σB= 0.001 .* abs.(B) # standard deviation for the perturbed matrix B°
σF= 0.001 .* abs.(F) # standard deviation for the perturbed matrix F°
σA= 0.001 .* abs.(A) # standard deviation for the perturbed matrix A°
σf = 0.001 .* abs.(f) # standard deviation for the perturbed vector f
σc = 0.001 .* abs.(c) # standard deviation for the perturbed vector c

N = 10000
s_res=zeros(length(processes),N);
d_res=zeros(length(boundaries),N);
x_res=zeros(length(boundaries),N);


In [13]:
for k ∈ 1:N

    modelᵦ = Model(CPLEX.Optimizer)
    set_silent(modelᵦ)
    𝚪ᵦ°= Matrix(𝐈-matformat(rand.(δ𝐁)))^-1 #FIXME: uncertainties
    ω°= rand.(TriangularDist.(ω.*0.8, ω.*1.2,ω))
    A°= rand.(Normal.(A, σA)) # A shouldn't change, 
    B°= rand.(Normal.(B, σB))
    F°= rand.(Normal.(F, σF))
    f°= rand.(Normal.(f, σf))
    c°= rand.(Normal.(c, σc))

    # Decision variable
    @variable(modelᵦ, 0 ≤ 𝐬ᵦ[1:length(processes)])

    # Expressions (equalities)
    @expression(modelᵦ, 𝐠ᵦ, B°*𝐬ᵦ)
    @expression(modelᵦ, 𝐝ᵦ, (Q*𝐠ᵦ)./ω°)
    @expression(modelᵦ, 𝐱ᵦ, 𝚪ᵦ * 𝐝ᵦ)

    # Expressions (Constraints on the decision variable)
    @constraint(modelᵦ, A° * 𝐬ᵦ .== f°)
    @constraint(modelᵦ, F° * 𝐬ᵦ .≤ c°)

    # Objective function: Minimising the state of control variables
    @objective(modelᵦ, Min, 𝐱ᵦ)    
    optimize!(modelᵦ)
    if !is_solved_and_feasible(modelᵦ)
        error("Solver did not find an optimal solution")
    end

    # extract values to store stochastic results
    d_res[:,k] = value.(𝐝ᵦ)
    s_res[:,k] = value.(𝐬ᵦ)
    x_res[:,k] = value.(𝐱ᵦ)
end
# solution_summary(modelᵦ)


In [14]:
fig, axs = subplots(3,3,figsize=(10,10),sharey=true)
for i in 1:9
    axs[i].hist(x_res[i,:], bins=100, alpha=0.5)
    axs[i].axvline(median(x_res[i,:]), color="red", linestyle="--", label="median")

    axs[i].set_title(catnames_ticks[i])
    axs[i].legend(frameon=false)
end
# display(plt.gcf())


In [15]:
fig, axs = subplots(3,3,figsize=(10,10))
for i in 1:9
    axs[i].hist(d_res[i,:], bins=100, alpha=0.5)
end
# display(plt.gcf())


In [16]:
fig, axs = subplots(4,4,figsize=(12,10))
for i in 1:9
    axs[i].hist(s_res[i,:], bins=100, alpha=0.5)
end
# display(plt.gcf())


### **3.1.4 Optimisation with informed by biophysical+human-mediated interactions**

In [17]:
modelₕ = Model(CPLEX.Optimizer);
set_silent(modelₕ)

@variable(modelₕ, 0≤ 𝐬ₕ[1:length(processes)])

@expression(modelₕ, 𝐠ₕ, B*𝐬ₕ)
@expression(modelₕ, 𝐝ₕ, (Q*𝐠ₕ)./ω)
@expression(modelₕ, 𝐱ₕ, 𝚪ₕ * 𝐝ₕ)

@objective(modelₕ, Min, 𝐱ₕ)
@constraint(modelₕ, A * 𝐬ₕ == f) 
# @constraint(modelₕ, F * 𝐬ₕ ≤ c)
optimize!(modelₕ);
# solution_summary(modelₕ)
DataFrame(hcat(catnames_ticks,[value.(i) for i ∈ [𝐝ₕ,𝐱ₕ]]...),["Boundaries","𝐝ₕ", "𝐱ₕ"])


Row,Boundaries,𝐝ₕ,𝐱ₕ
,Any,Any,Any
1,Climate change Energy imbalance,6.75738,18.8
2,Ocean acidification,1.97176,9.5582
3,Atmospheric aerosol loading,1.47063,-6.28762
4,Freshwater use,11.5038,17.5222
5,Biogeochemical flows-P,2.62535,11.4299
6,Biogeochemical flows-N,2.89673,11.4299
7,Stratospheric ozone depletion,0.189737,5.86988
8,Land-system change,0.00215625,32.681
9,Biosphere Integrity,8.68749,18.6784


In [18]:
a=value.(𝐱ₕ)


9-element Vector{Float64}:
 18.800022439155946
  9.558203980871276
 -6.2876176428685095
 17.522203153856026
 11.429861060618665
 11.429861060618665
  5.869880052709068
 32.68095982080536
 18.678388884239876

In [19]:
# b,c=pbplot(a,categories=catnames_ticks,scale=4)
# display(b)


In [20]:
DataFrame(hcat(processes,[value.(i) for i ∈ [𝐬ₕ]]...),["processes","𝐬"])


Row,processes,𝐬
,Any,Any
1,Rice factory,1.0
2,Rice farming,0.0
3,Low-nitrogen,1.15
4,Rice husk boiler,2.2
5,Natural gas boiler,0.0
6,Wood pellet boiler,0.0
7,Rice husk collection 1,0.506
8,Rice husk collection 2,0.0
9,Rice husk collection 3,0.0


## **3.2. Results**

### **3.2.1 System analysis**

In [21]:
S=[𝐬, 𝐬ᶜᶜ,𝐬ᵦ,𝐬ₕ]
DataFrame(hcat([processes,[value.(i) for i ∈ S]...]...),["processes","s", "sᶜᶜ", "sᵦ","sₕ"])



Row,processes,s,sᶜᶜ,sᵦ,sₕ
,Any,Any,Any,Any,Any
1,Rice factory,1.0,1.0,1.00152,1.0
2,Rice farming,0.0,1.15,0.0,0.0
3,Low-nitrogen,1.15,0.0,1.15194,1.15
4,Rice husk boiler,2.2,2.2,2.17061,2.2
5,Natural gas boiler,0.0,0.0,0.0,0.0
6,Wood pellet boiler,0.0,0.0,0.0354433,0.0
7,Rice husk collection 1,0.506,0.506,0.100207,0.506
8,Rice husk collection 2,0.0,0.0,0.0999668,0.0
9,Rice husk collection 3,0.0,0.0,0.0999256,0.0


### **3.2.2 Direct human impact**

In [22]:
S=[𝐝,𝐝ᶜᶜ,𝐝ᵦ,𝐝ₕ]
DataFrame(hcat([catnames_ticks,[value.(i) for i ∈ S]...]...),["processes","𝐝","𝐝ᶜᶜ","𝐝ᵦ","𝐝ₕ"])


Row,processes,𝐝,𝐝ᶜᶜ,𝐝ᵦ,𝐝ₕ
,Any,Any,Any,Any,Any
1,Climate change Energy imbalance,6.75738,5.13588,7.2246,6.75738
2,Ocean acidification,1.97176,7.46495,2.34945,1.97176
3,Atmospheric aerosol loading,1.47063,46.6898,1.60893,1.47063
4,Freshwater use,11.5038,0.00128397,11.6008,11.5038
5,Biogeochemical flows-P,2.62535,0.513588,2.86122,2.62535
6,Biogeochemical flows-N,2.89673,0.129367,2.57181,2.89673
7,Stratospheric ozone depletion,0.189737,0.354199,0.178681,0.189737
8,Land-system change,0.00215625,0.205435,0.002119,0.00215625
9,Biosphere Integrity,8.68749,0.513588,8.38905,8.68749


### **3.2.3 State of plenatary bondaries**

In [23]:
S=[𝐱,𝐱ᶜᶜ,𝐱ᵦ,𝐱ₕ]
DataFrame(hcat([catnames_ticks,[value.(i) for i ∈ S]...]...),["processes","𝐱","𝐱ᶜᶜ","𝐱ᵦ","𝐱ₕ"])


Row,processes,𝐱,𝐱ᶜᶜ,𝐱ᵦ,𝐱ₕ
,Any,Any,Any,Any,Any
1,Climate change Energy imbalance,6.75738,5.13588,12.0243,18.8
2,Ocean acidification,1.97176,7.46495,8.6722,9.5582
3,Atmospheric aerosol loading,1.47063,46.6898,-5.12467,-6.28762
4,Freshwater use,11.5038,0.00128397,16.7882,17.5222
5,Biogeochemical flows-P,2.62535,0.513588,10.696,11.4299
6,Biogeochemical flows-N,2.89673,0.129367,10.696,11.4299
7,Stratospheric ozone depletion,0.189737,0.354199,-1.14399,5.86988
8,Land-system change,0.00215625,0.205435,6.30925,32.681
9,Biosphere Integrity,8.68749,0.513588,16.4432,18.6784


# **4. How to construct a Technology choice model**

## **4.1 From linear algebra**

We can get the matrices as follows

In [24]:
𝐀=Technosphere!().Matrix
𝐁=Biosphere!().Matrix
Q=Characterisation!().Matrix


UndefVarError: UndefVarError: `projectᵦ` not defined

# **References**